<a href="https://colab.research.google.com/github/AyushXRane/Alzheimer-MRI-Classifier/blob/main/Alzheimer's_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/Project/Alzheimers.zip'

Streaming output truncated to the last 5000 lines.
  inflating: Data/Very mild Dementia/OAS1_0247_MR1_mpr-4_102.jpg  
  inflating: Data/Very mild Dementia/OAS1_0247_MR1_mpr-4_103.jpg  
  inflating: Data/Very mild Dementia/OAS1_0247_MR1_mpr-4_104.jpg  
  inflating: Data/Very mild Dementia/OAS1_0247_MR1_mpr-4_105.jpg  
  inflating: Data/Very mild Dementia/OAS1_0247_MR1_mpr-4_106.jpg  
  inflating: Data/Very mild Dementia/OAS1_0247_MR1_mpr-4_107.jpg  
  inflating: Data/Very mild Dementia/OAS1_0247_MR1_mpr-4_108.jpg  
  inflating: Data/Very mild Dementia/OAS1_0247_MR1_mpr-4_109.jpg  
  inflating: Data/Very mild Dementia/OAS1_0247_MR1_mpr-4_110.jpg  
  inflating: Data/Very mild Dementia/OAS1_0247_MR1_mpr-4_111.jpg  
  inflating: Data/Very mild Dementia/OAS1_0247_MR1_mpr-4_112.jpg  
  inflating: Data/Very mild Dementia/OAS1_0247_MR1_mpr-4_113.jpg  
  inflating: Data/Very mild Dementia/OAS1_0247_MR1_mpr-4_114.jpg  
  inflating: Data/Very mild Dementia/OAS1_0247_MR1_mpr-4_115.jpg  
  inflating

In [ ]:
import os
print(os.getcwd())


/content


In [ ]:
!ls './Data/'



'Mild Dementia'  'Moderate Dementia'  'Non Demented'  'Very mild Dementia'


In [ ]:
import os
file_path = './Data/'

files = os.listdir(file_path)
print(files)



['Non Demented', 'Moderate Dementia', 'Mild Dementia', 'Very mild Dementia']


In [ ]:
import os
import random

# Define the paths to the two class folders
classes_to_reduce = {
    "Class1": "/content/Data/Non Demented",  # Replace with your actual folder path
    "Class2": "/content/Data/Very mild Dementia",  # Replace with your actual folder path
}

# Specify the target number of images to keep in each class
target_size = 5000  # Adjust this to the desired number of images to keep

# Iterate over each class
for class_name, folder_path in classes_to_reduce.items():
    # List all files in the class folder
    files = os.listdir(folder_path)
    num_files = len(files)
    print(f"{class_name}: {num_files} images found.")

    # Check if the class has more images than the target size
    if num_files > target_size:
        # Calculate how many files to remove
        excess = num_files - target_size
        print(f"Removing {excess} files from {class_name}...")

        # Randomly select files to delete
        files_to_delete = random.sample(files, excess)

        # Remove the selected files
        for file_name in files_to_delete:
            file_path = os.path.join(folder_path, file_name)
            os.remove(file_path)

        print(f"Finished removing {excess} images from {class_name}. Remaining files: {len(os.listdir(folder_path))}")
    else:
        print(f"No images removed from {class_name}; already meets target size.")


Class1: 67222 images found.
Removing 62222 files from Class1...
Finished removing 62222 images from Class1. Remaining files: 5000
Class2: 13725 images found.
Removing 8725 files from Class2...
Finished removing 8725 images from Class2. Remaining files: 5000


In [ ]:
print({len(os.listdir('./Data/Non Demented'))})
print({len(os.listdir('./Data/Mild Dementia'))})
print({len(os.listdir('./Data/Moderate Dementia'))})
print({len(os.listdir('./Data/Very mild Dementia'))})


{5000}
{5002}
{488}
{5000}


In [ ]:
import cv2
import numpy as np
import os
from sklearn.preprocessing import OneHotEncoder

#Function to load images from folder
def load_images_from_folder(folder, category):
    images = []
    labels = []
    if not os.path.exists(folder):
        raise ValueError(f"Directory does not exist: {folder}")
    print(f"Loading images from {folder}...")
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, (150, 150))  # Resize images
            img = img.astype(np.float32) / 255.0  # Normalize pixel values
            images.append(img)
            labels.append(category)
    if not images or not labels:
        raise ValueError("No images or labels found in the specified directory.")
    return np.array(images), np.array(labels)

#Load images from each directory
mild_demented_dir = './Data/Mild Dementia'
moderate_demented_dir = './Data/Moderate Dementia'
non_demented_dir = './Data/Non Demented'
very_mild_demented_dir = './Data/Very mild Dementia'

#Load the images from each directory with corresponding labels
mild_demented_data, mild_demented_labels = load_images_from_folder(mild_demented_dir, 'mild')
moderate_demented_data, moderate_demented_labels = load_images_from_folder(moderate_demented_dir, 'moderate')
non_demented_data, non_demented_labels = load_images_from_folder(non_demented_dir, 'non')
very_mild_demented_data, very_mild_demented_labels = load_images_from_folder(very_mild_demented_dir, 'very_mild')

#Concatenate data and labels
data = np.concatenate((mild_demented_data, moderate_demented_data, non_demented_data, very_mild_demented_data))
labels = np.concatenate((mild_demented_labels, moderate_demented_labels, non_demented_labels, very_mild_demented_labels))

#One-Hot Encoding the labels
one_hot_encoder = OneHotEncoder(sparse_output=False)
labels_encoded = one_hot_encoder.fit_transform(labels.reshape(-1, 1))

print(data.shape)
print(labels_encoded.shape)


Loading images from ./Data/Mild Dementia...
Loading images from ./Data/Moderate Dementia...
Loading images from ./Data/Non Demented...
Loading images from ./Data/Very mild Dementia...
(15490, 150, 150, 3)
(15490, 4)


In [ ]:
print(np.unique(labels))

['mild' 'moderate' 'non' 'very_mild']


In [ ]:
print("Encoded Labels:", labels_encoded)
print("Unique Encoded Values:", np.unique(labels_encoded))

Encoded Labels: [[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]]
Unique Encoded Values: [0. 1.]


In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data, train_labels, test_labels = train_test_split(data, labels_encoded, test_size=0.2, random_state=42)



In [ ]:
from keras.models import Sequential
from keras import layers


model = Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(4, activation='softmax')
])

#batch normalization after each CV layer
#droupout after each maxpooling, ex model.add(Dropout(0.25))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, MaxPooling2D, Dropout, Flatten, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

for layer in base_model.layers:
    layer.trainable = False

# Unfreeze the last 10 layers
for layer in base_model.layers[-15:]:
    layer.trainable = True


model_preTrained = Sequential([
    base_model,
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(4, activation='softmax')  # Output layer for 4 classes
])




94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#Train the model
history = model.fit(train_data, train_labels, validation_split=0.2, epochs=10, batch_size=32)

#Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(test_data, test_labels)
print("Test accuracy:", test_accuracy)

Epoch 1/10
310/310 ━━━━━━━━━━━━━━━━━━━━ 229s 725ms/step - accuracy: 0.4754 - loss: 4.9614 - val_accuracy: 0.3235 - val_loss: 3.9281
Epoch 2/10
310/310 ━━━━━━━━━━━━━━━━━━━━ 220s 708ms/step - accuracy: 0.7557 - loss: 0.5940 - val_accuracy: 0.4421 - val_loss: 1.7467
Epoch 3/10
310/310 ━━━━━━━━━━━━━━━━━━━━ 223s 720ms/step - accuracy: 0.8430 - loss: 0.3828 - val_accuracy: 0.9088 - val_loss: 0.2845
Epoch 4/10
310/310 ━━━━━━━━━━━━━━━━━━━━ 221s 712ms/step - accuracy: 0.9126 - loss: 0.2531 - val_accuracy: 0.9282 - val_loss: 0.2144
Epoch 5/10
310/310 ━━━━━━━━━━━━━━━━━━━━ 217s 699ms/step - accuracy: 0.9363 - loss: 0.1992 - val_accuracy: 0.9359 - val_loss: 0.2042
Epoch 6/10
310/310 ━━━━━━━━━━━━━━━━━━━━ 216s 698ms/step - accuracy: 0.9528 - loss: 0.1566 - val_accuracy: 0.8370 - val_loss: 0.4717
Epoch 7/10
310/310 ━━━━━━━━━━━━━━━━━━━━ 219s 706ms/step - accuracy: 0.9627 - loss: 0.1265 - val_accuracy: 0.9375 - val_loss: 0.1968
Epoch 8/10
310/310 ━━━━━━━━━━━━━━━━━━━━ 219s 707ms/step - accuracy: 0.9665 -

In [ ]:


from sklearn.metrics import classification_report, confusion_matrix
predictions = model.predict(test_data)
predicted_classes = np.argmax(predictions, axis=1)

true_classes = np.argmax(test_labels, axis=1)  # The true classes
report = classification_report(true_classes, predicted_classes, target_names=one_hot_encoder.categories_[0])
print("Classification Report:")
print(report)

conf_matrix = confusion_matrix(true_classes, predicted_classes)
print("Confusion Matrix:")
print(conf_matrix)

97/97 ━━━━━━━━━━━━━━━━━━━━ 12s 125ms/step
Classification Report:
              precision    recall  f1-score   support

        mild       1.00      0.99      0.99      1018
    moderate       0.96      0.93      0.94       100
         non       0.94      0.97      0.96      1012
   very_mild       0.98      0.95      0.96       968

    accuracy                           0.97      3098
   macro avg       0.97      0.96      0.96      3098
weighted avg       0.97      0.97      0.97      3098

Confusion Matrix:
[[1011    0    6    1]
 [   0   93    7    0]
 [   4    4  986   18]
 [   1    0   49  918]]


In [ ]:
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
import numpy as np

def create_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(4, activation='softmax')  ])

    model.compile(optimizer=Adam(learning_rate=1e-4),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold_accuracies = []

for fold, (train_index, val_index) in enumerate(kf.split(data)):
    print(f"Training on fold {fold + 1}...")

    X_train, X_val = data[train_index], data[val_index]
    y_train, y_val = labels_encoded[train_index], labels_encoded[val_index]

    model = create_model()

    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, batch_size=32, verbose=1)

    val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
    print(f"Fold {fold + 1} Accuracy: {val_acc:.4f}")
    fold_accuracies.append(val_acc)

print(f"\nCross-Validation Accuracies: {fold_accuracies}")
print(f"Mean Accuracy: {np.mean(fold_accuracies):.4f}")
print(f"Standard Deviation: {np.std(fold_accuracies):.4f}")



Training on fold 1...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
388/388 ━━━━━━━━━━━━━━━━━━━━ 320s 814ms/step - accuracy: 0.5563 - loss: 1.1412 - val_accuracy: 0.3399 - val_loss: 2.0737
Epoch 2/5
388/388 ━━━━━━━━━━━━━━━━━━━━ 290s 747ms/step - accuracy: 0.8338 - loss: 0.4247 - val_accuracy: 0.7085 - val_loss: 0.6323
Epoch 3/5
388/388 ━━━━━━━━━━━━━━━━━━━━ 309s 796ms/step - accuracy: 0.9154 - loss: 0.2407 - val_accuracy: 0.8567 - val_loss: 0.3640
Epoch 4/5
388/388 ━━━━━━━━━━━━━━━━━━━━ 276s 712ms/step - accuracy: 0.9462 - loss: 0.1645 - val_accuracy: 0.8954 - val_loss: 0.2695
Epoch 5/5
388/388 ━━━━━━━━━━━━━━━━━━━━ 306s 788ms/step - accuracy: 0.9645 - loss: 0.1115 - val_accuracy: 0.9555 - val_loss: 0.1353
Fold 1 Accuracy: 0.9555
Training on fold 2...
Epoch 1/5
388/388 ━━━━━━━━━━━━━━━━━━━━ 296s 753ms/step - accuracy: 0.5310 - loss: 1.2712 - val_accuracy: 0.3354 - val_loss: 6.4454
Epoch 2/5
388/388 ━━━━━━━━━━━━━━━━━━━━ 317s 817ms/step - accuracy: 0.8313 - loss: 0.4377 - val_accuracy: 0.6772 - val_loss: 0.7287
Epoch 3/5
388/388 ━━━━━━━━━━━━━━━━━━━

In [ ]:
model_preTrained.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])


history_preTrained = model_preTrained.fit(train_data, train_labels, validation_split=0.2, epochs=10, batch_size=32)
test_loss, test_accuracy = model_preTrained.evaluate(test_data, test_labels)
print("Test accuracy:", test_accuracy)

from sklearn.metrics import classification_report, confusion_matrix
predictions = model_preTrained.predict(test_data)
predicted_classes = np.argmax(predictions, axis=1)

true_classes = np.argmax(test_labels, axis=1)  # The true classes
report = classification_report(true_classes, predicted_classes, target_names=one_hot_encoder.categories_[0])
print("Classification Report:")
print(report)

conf_matrix = confusion_matrix(true_classes, predicted_classes)
print("Confusion Matrix:")
print(conf_matrix)

Epoch 1/10
310/310 ━━━━━━━━━━━━━━━━━━━━ 360s 1s/step - accuracy: 0.5382 - loss: 1.1646 - val_accuracy: 0.6995 - val_loss: 0.7676
Epoch 2/10
310/310 ━━━━━━━━━━━━━━━━━━━━ 352s 1s/step - accuracy: 0.7374 - loss: 0.6207 - val_accuracy: 0.7858 - val_loss: 0.5215
Epoch 3/10
310/310 ━━━━━━━━━━━━━━━━━━━━ 339s 1s/step - accuracy: 0.7873 - loss: 0.5029 - val_accuracy: 0.8431 - val_loss: 0.3875
Epoch 4/10
310/310 ━━━━━━━━━━━━━━━━━━━━ 335s 1s/step - accuracy: 0.8386 - loss: 0.3942 - val_accuracy: 0.8657 - val_loss: 0.3457
Epoch 5/10
310/310 ━━━━━━━━━━━━━━━━━━━━ 341s 1s/step - accuracy: 0.8711 - loss: 0.3191 - val_accuracy: 0.8499 - val_loss: 0.3541
Epoch 6/10
310/310 ━━━━━━━━━━━━━━━━━━━━ 337s 1s/step - accuracy: 0.8891 - loss: 0.2704 - val_accuracy: 0.8608 - val_loss: 0.3451
Epoch 7/10
310/310 ━━━━━━━━━━━━━━━━━━━━ 338s 1s/step - accuracy: 0.9115 - loss: 0.2259 - val_accuracy: 0.9185 - val_loss: 0.2218
Epoch 8/10
310/310 ━━━━━━━━━━━━━━━━━━━━ 338s 1s/step - accuracy: 0.9318 - loss: 0.1750 - val_accu

In [ ]:
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, MaxPooling2D, Dropout, Flatten, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
import numpy as np

# Load the ResNet50 base model (excluding the top layers)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Freeze all layers initially
for layer in base_model.layers:
    layer.trainable = False

# Unfreeze the last 15 layers for fine-tuning
for layer in base_model.layers[-15:]:
    layer.trainable = True

# Function to create the model
def create_model():
    model = Sequential([
        base_model,
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(4, activation='softmax')  # Output layer for 4 classes
    ])

    model.compile(optimizer=Adam(learning_rate=1e-4),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold_accuracies = []

for fold, (train_index, val_index) in enumerate(kf.split(data)):
    print(f"Training on fold {fold + 1}...")

    X_train, X_val = data[train_index], data[val_index]
    y_train, y_val = labels_encoded[train_index], labels_encoded[val_index]

    model = create_model()

    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, batch_size=32, verbose=1)

    val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
    print(f"Fold {fold + 1} Accuracy: {val_acc:.4f}")
    fold_accuracies.append(val_acc)

print(f"\nCross-Validation Accuracies: {fold_accuracies}")
print(f"Mean Accuracy: {np.mean(fold_accuracies):.4f}")
print(f"Standard Deviation: {np.std(fold_accuracies):.4f}")


Training on fold 1...
Epoch 1/5
388/388 ━━━━━━━━━━━━━━━━━━━━ 450s 1s/step - accuracy: 0.5325 - loss: 1.1959 - val_accuracy: 0.6362 - val_loss: 0.7939
Epoch 2/5
388/388 ━━━━━━━━━━━━━━━━━━━━ 416s 1s/step - accuracy: 0.7463 - loss: 0.6025 - val_accuracy: 0.8144 - val_loss: 0.4713
Epoch 3/5
388/388 ━━━━━━━━━━━━━━━━━━━━ 418s 1s/step - accuracy: 0.8191 - loss: 0.4339 - val_accuracy: 0.7750 - val_loss: 0.5433
Epoch 4/5
388/388 ━━━━━━━━━━━━━━━━━━━━ 417s 1s/step - accuracy: 0.8637 - loss: 0.3372 - val_accuracy: 0.7140 - val_loss: 0.7237
Epoch 5/5
388/388 ━━━━━━━━━━━━━━━━━━━━ 419s 1s/step - accuracy: 0.8799 - loss: 0.3035 - val_accuracy: 0.8857 - val_loss: 0.2822
Fold 1 Accuracy: 0.8857
Training on fold 2...
Epoch 1/5
388/388 ━━━━━━━━━━━━━━━━━━━━ 441s 1s/step - accuracy: 0.7711 - loss: 0.6244 - val_accuracy: 0.8644 - val_loss: 0.3342
Epoch 2/5
388/388 ━━━━━━━━━━━━━━━━━━━━ 446s 1s/step - accuracy: 0.8914 - loss: 0.2800 - val_accuracy: 0.7498 - val_loss: 0.9183
Epoch 3/5
388/388 ━━━━━━━━━━━━━━━━━━